# 切换所处目录位置
## 核心概念
*目录树结构*：
    文件系统是树状结构，根目录 (/) 是起点
*进程工作目录 (CWD - Current Working Directory)*：
    每个进程（包括 shell）都有一个当前工作目录，本质是内核维护的目录inode指针
*绝对路径和相对路径*：
    绝对路径是从根目录出发的完整文件路径，而相对路径是从上层文件位置出发的，相对于上层的文件路径
## cd，pwd
*cd的核心作用是改变当前 shell 进程的工作目录（CWD）*
*pwd的核心作用是打印当前CWD的绝对路径*
> 基本语法
cd /path/to/directory/
pwd
> 参数
cd `-`[上一个目录位置] `..`[上一级目录位置] `~`[当前登录用户的家目录位置]

# 查看文件目录与文件具体信息
## 核心概念
*inode*:
    文件系统内部用于描述和管理一个文件（包括目录、设备文件等一切“文件”）元数据的数据结构。每个文件/目录有且仅有一个唯一的 inode。
*inode存储*：
    文件类型 (st_mode): `普通文件 (-)`、`目录 (d)`、`符号链接 (l)`、块设备 (b)、字符设备 (c)、管道 (p)、套接字 (s)。
    权限 (st_mode): `读 (r)、写 (w)、执行 (x) 权限位`，针对文件`所有者 (Owner)、所属组 (Group) 和其他用户 (Others)。`
    链接数 (st_nlink): 指向该 inode 的`硬`链接总数。
    所有者 (st_uid): 文件`所有者的用户 ID`。
    所属组 (st_gid): 文件`所属组的组 ID`。
    大小 (st_size): 文件内容的字节大小（对于目录，通常指其目录项列表的大小）。
    时间戳:
        `访问时间 (st_atime)`: 文件内容最后被读取的时间。
        `修改时间 (st_mtime)`: 文件内容最后被修改的时间。
        `变更时间 (st_ctime)`: 文件 inode 元数据（权限、所有者、链接数等）最后被改变的时间（在 Linux 中）。
        `创建时间 (st_birthtime)`: 文件创建时间（并非所有文件系统都支持）。
    数据块指针： 指向磁盘上存储文件实际内容的物理块位置（对用户透明）。
*目录*：
    将人类可读的文件名与系统内部标识文件的 inode 编号关联起来的`映射表`
*符号链接*
    提供一种指向另一个文件的“快捷方式”或“别名”。删除符号链接不影响目标文件；删除目标文件会导致符号链接“悬空”。
## ls,stat,tree
*ls，stat，tree本质都是inode元数据的提取器*
- ls：`输入`：目录路径 -> `输出`：文件名 + inode元数据摘要 (类型, 权限, 链接数, 所有者, 组, 大小, 时间)
- stat:`输入`：文件/目录路径 ->` 输出`：完整的inode元数据
- tree:`输入`:目录路径 (根) -> `输出：`递归的目录树 + 文件/目录名
[区别：ls输出的inode信息是stat的子集，例如ls只支持mt这一个时间戳，而stat是完整的]
*参数主要改变对inode元数据的提取详细程度,与输出格式*
>基本语法：
ls -[option] /path/to/directory/
stat -[option] /path/to/directory/
tree -[option] /path/to/directory/
>参数
ls -`l`[详细列表如权限、大小、时间]-`a`[显示隐藏文件以.开头]-`h`[人类可读的文件大小]-`S`[按文件大小排序]-`t`[按修改时间排序]-`r`[反向排序]-`ld`[输入为文件夹时需添加]
stat列出了完整的inode，因此无需额外参数支持
tree-`d`[仅列出文件夹]-`L number`[限制查找深度为number层 ]

# 搜索并筛选文件
## find，locate
*find是基于VFS的深度遍历工具，直接交互内核获取实时数据，灵活但资源敏感。*
*locate利用预构建索引( /var/lib/slocate)的查询工具，牺牲实时性换取毫秒级响应，适合快速定位。*
- find：`输入`：起点路径,搜索条件 -> `输出`：匹配文件的完整路径（到标准输出）
- locate:`输入`：搜索条件 ->  `输出`：匹配文件的完整路径（到标准输出）
> 基本语法：
find /path/to/directory/ [option]
locate  [option]
> 参数：
find `-name` `-iname` `-type` `-mtime` `-size` `-empty` `-delelt` -perm -o -not -exec -   